# Przykładowy agent rezerwacji hotelu i lotu

To rozwiązanie pomoże Ci zarezerwować bilety lotnicze oraz hotel. Scenariusz dotyczy podróży z Londynu Heathrow LHR 20 lutego 2024 do Nowego Jorku JFK z powrotem 27 lutego 2025, lot w klasie ekonomicznej wyłącznie liniami British Airways. Chcę zatrzymać się w hotelu Hilton w Nowym Jorku, proszę o podanie kosztów lotu i hotelu.


# Zainicjuj usługę Azure AI Agent i uzyskaj informacje konfiguracyjne z **.env**

### **.env** 

Utwórz plik .env 

**.env** zawiera ciąg połączenia do usługi Azure AI Agent, model używany przez AOAI oraz odpowiednie API wyszukiwania Google, ENDPOINT itp.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nazwa wdrożenia modelu usługi Azure AI Agent"

[**NOTE**] Potrzebujesz modelu z limitem 100,000 (Tokeny na minutę) oraz limitem 600 (Żądania na minutę).

  Model można uzyskać w Azure AI Foundry - Model i Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Ciąg połączenia projektu usługi Azure AI Agent"

  Ciąg połączenia projektu można znaleźć w przeglądzie projektu na ekranie portalu AI ​​Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Twój klucz API wyszukiwania SERPAPI"
- **SERPAPI_SEARCH_ENDPOINT** = "Twój endpoint wyszukiwania SERPAPI"

Aby uzyskać nazwę wdrożenia modelu i ciąg połączenia projektu usługi Azure AI Agent, musisz utworzyć usługę Azure AI Agent. Zaleca się użycie [tego szablonu](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) do bezpośredniego utworzenia usługi (***Uwaga:*** Usługa Azure AI Agent jest obecnie dostępna w ograniczonych regionach. Zaleca się zapoznanie się z [tym linkiem](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) w celu ustawienia regionu).

Agent musi mieć dostęp do SERPAPI. Zaleca się rejestrację za pomocą [tego linku](https://serpapi.com/searches). Po rejestracji możesz uzyskać unikalny klucz API oraz ENDPOINT.


# Konfiguracja

Aby uruchomić ten notebook, upewnij się, że zainstalowałeś wymagane biblioteki, uruchamiając `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Twoja wersja Semantic Kernel powinna wynosić co najmniej 1.27.2.


Załaduj ustawienia i zasoby pliku .env, upewnij się, że dodałeś swoje klucze i ustawienia oraz utworzyłeś lokalny plik .env.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Zaloguj się do Azure

Teraz musisz zalogować się do Azure. Otwórz terminal i uruchom następujące polecenie:

```bash
az login
```

To polecenie poprosi Cię o wprowadzenie danych uwierzytelniających do Azure, co umożliwi prawidłowe działanie usługi Azure AI Agent.


# Wyjaśnienie:
To jest zmienna, która przechowuje klucz API umożliwiający dostęp do usługi API SERP (Search Engine Results Page). Klucz API to unikalny identyfikator używany do uwierzytelniania żądań powiązanych z Twoim kontem.

Cel: Celem tej linii jest przechowywanie klucza API w zmiennej, aby można go było używać do uwierzytelniania żądań do usługi API SERP. Klucz API jest wymagany, aby uzyskać dostęp do usługi i wykonywać wyszukiwania.
Jak uzyskać klucz API SERP: Aby uzyskać klucz API SERP, wykonaj następujące ogólne kroki na stronie https://serpapi.com (dokładne kroki mogą się różnić w zależności od konkretnej usługi API SERP, z której korzystasz):

Wybierz usługę API SERP: Istnieje kilka dostępnych usług API SERP, takich jak SerpAPI, Google Custom Search JSON API i inne. Wybierz tę, która najlepiej odpowiada Twoim potrzebom.

Zarejestruj się: Przejdź na stronę wybranej usługi API SERP i załóż konto. Może być konieczne podanie podstawowych informacji oraz weryfikacja adresu e-mail.

Utwórz klucz API: Po rejestracji zaloguj się na swoje konto i przejdź do sekcji API lub panelu sterowania. Poszukaj opcji utworzenia lub wygenerowania nowego klucza API.
Skopiuj klucz API do swojego pliku .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Wyjaśnienie:
BASE_URL: To zmienna, która przechowuje podstawowy URL dla punktu końcowego API SERP. Nazwa zmiennej BASE_URL jest konwencją wskazującą, że ten URL jest punktem wyjścia do wykonywania żądań API.  
'https://serpapi.com/search':  

To rzeczywisty ciąg URL przypisany do zmiennej BASE_URL. Reprezentuje on punkt końcowy do wykonywania zapytań wyszukiwania za pomocą API SERP.

# Cel:
Celem tej linii jest zdefiniowanie stałej, która przechowuje podstawowy URL dla API SERP. Ten URL będzie używany jako punkt wyjścia do budowania żądań API w celu wykonywania operacji wyszukiwania.

# Zastosowanie:
Definiując podstawowy URL w zmiennej, możesz łatwo go ponownie używać w całym kodzie, gdy tylko zajdzie potrzeba wykonania żądań do API SERP. Dzięki temu Twój kod staje się bardziej przejrzysty i łatwiejszy w utrzymaniu, a także zmniejsza się ryzyko błędów wynikających z wielokrotnego wpisywania URL-a w różnych miejscach. Obecny przykład to https://serpapi.com/search?engine=bing, który korzysta z API wyszukiwania Bing. Różne API można wybrać na stronie https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Wyjaśnienie:

Tutaj znajduje się kod Twojej wtyczki.

Definicja klasy: `class BookingPlugin`: Definiuje klasę o nazwie BookingPlugin, która zawiera metody do rezerwacji hoteli i lotów.

Metoda rezerwacji hotelu:

- `@kernel_function(description="booking hotel")`: Dekorator opisujący funkcję jako funkcję jądra do rezerwacji hoteli.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definiuje metodę do rezerwacji hoteli z adnotowanymi parametrami i typem zwracanym.

Metoda tworzy słownik parametrów dla żądania rezerwacji hotelu i wysyła żądanie GET do API SERP. Sprawdza status odpowiedzi i zwraca właściwości hotelu w przypadku powodzenia lub None, jeśli żądanie się nie powiodło.

Metoda rezerwacji lotu:

- `@kernel_function(description="booking flight")`: Dekorator opisujący funkcję jako funkcję jądra do rezerwacji lotów.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definiuje metodę do rezerwacji lotów z adnotowanymi parametrami i typem zwracanym.

Metoda tworzy słowniki parametrów dla żądań dotyczących lotu w jedną stronę i powrotnego, a następnie wysyła żądania GET do API SERP. Sprawdza status odpowiedzi i dodaje informacje o locie do ciągu wynikowego w przypadku powodzenia lub wyświetla komunikat o błędzie, jeśli żądanie się nie powiodło. Metoda zwraca ciąg wynikowy zawierający informacje o locie.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Wyjaśnienie:
Importowanie modułów: Importowanie niezbędnych modułów dla poświadczeń Azure, agenta AI, treści wiadomości czatu, roli autora oraz dekoratora funkcji jądra.

Asynchroniczny menedżer kontekstu: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Ustawia asynchroniczny menedżer kontekstu do obsługi poświadczeń Azure i tworzenia klienta agenta AI.

Nazwa agenta i instrukcje:
- `AGENT_NAME = "BookingAgent"`: Definiuje nazwę agenta.
- `AGENT_INSTRUCTIONS = """..."""`: Dostarcza szczegółowe instrukcje dla agenta dotyczące obsługi żądań rezerwacji.

Tworzenie definicji agenta: `agent_definition = await client.agents.create_agent(...)`: Tworzy definicję agenta z określonym modelem, nazwą i instrukcjami.

Tworzenie agenta AzureAI: `agent = AzureAIAgent(...)`: Tworzy agenta AzureAI za pomocą klienta, definicji agenta oraz zdefiniowanej wtyczki.

Tworzenie wątku: `thread: AzureAIAgentThread | None = None`: Tworzy wątek dla agenta. Nie jest wymagane wcześniejsze utworzenie wątku - jeśli wartość `None` zostanie podana, nowy wątek zostanie utworzony podczas pierwszego wywołania i zwrócony jako część odpowiedzi.

Dane wejściowe użytkownika: `user_inputs = ["..."]`: Definiuje listę danych wejściowych użytkownika, które agent ma przetworzyć.

W bloku finally usuń wątek i agenta, aby zwolnić zasoby.


# Uwierzytelnianie

Klasa `DefaultAzureCredential` jest częścią Azure SDK dla Pythona. Zapewnia domyślny sposób uwierzytelniania w usługach Azure. Próbuje uwierzytelniać za pomocą różnych metod w określonej kolejności, takich jak zmienne środowiskowe, zarządzana tożsamość i poświadczenia Azure CLI.

Operacje asynchroniczne: Moduł aio wskazuje, że klasa DefaultAzureCredential obsługuje operacje asynchroniczne. Oznacza to, że możesz używać jej z asyncio do wykonywania nieblokujących żądań uwierzytelniania.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż staramy się zapewnić dokładność, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego języku źródłowym powinien być uznawany za autorytatywne źródło. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
